# KNAPSACK PROBLEM

list of things
1. the problem - capacitated hvrp
1. Classical overview and algorithem explanation
1. quantum advantages
1. the uantum algorithem
1. benchmarking  -  Comparing qaoa solution with classical solver
1. conclusions and future work


Things to write about 
1. DOCPLEX https://arxiv.org/pdf/2202.13125.pdf
1. COBYL Ahttps://arxiv.org/pdf/2202.13125.pdf
1. qaoa
1. 


1. in this main notbook -
1. importing relavent building blocks file for classical, and for quantum
1. setting up values, weighs and other parameters
1.  calling classical function, 
1.  calling quantum function
1. benchmarcking - plotting



$$
Ha = A * \sum\limits_{v}\left(\left( \sum\limits_{k=0}^{M^v-1} (2^k z_k^v + (Q^v + 1 - 2^{M^v}  ) z_M^v\right) -  \sum\limits_{a,i}q_i y_{ia}^v\right)^2


$$



$$Hb = -B \sum\limits_i^N(c_i x_i)$$